# Exploratory Analysis on Airplane Data

### Business Problem

Your company is expanding in to new industries to diversify its portfolio. Specifically, they are interested in purchasing and operating airplanes for commercial and private enterprises, but do not know anything about the potential risks of aircraft.

You are charged with determining which aircraft are the lowest risk for the company to start this new business endeavor.

You must then translate your findings into actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase.

In [1]:
# Switch to determine whether running the notebook will generate new csvs (and overwrite previous versions) 
generate_csvs = 1

In [2]:
#Import libraries
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
warnings.filterwarnings('ignore')

## Preview the Available Data

In [3]:
#This will be the main DataFrame used to analyze our business problem
df = pd.read_csv('data/Aviation_Data.csv')
print(f"Original shape: {df.shape}")
df.drop_duplicates(keep='first', inplace=True)

print(f"Shape after dropping duplicate rows: {df.shape}")
df.head()

Original shape: (90348, 31)
Shape after dropping duplicate rows: (88958, 31)


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [4]:
#drop columns with more than 20% nans 
nan_cols = list(df.loc[:,df.isna().sum()/len(df) > .2].columns) 

#drop uninformative columns
nan_cols.extend(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Injury.Severity', 'Registration.Number', 'Report.Status', 'Publication.Date'])

# get a list of columns to keep
non_nan_cols = [x for x in df.columns if x not in nan_cols]

# re-add the aircraft category column for later cleaning
non_nan_cols.append('Aircraft.Category')
                    
# remove columns from df
df = df[non_nan_cols].copy()
df.head()

,Event.Date,Location,Country,Aircraft.damage,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Aircraft.Category
0,1948-10-24,"MOOSE CREEK, ID",United States,Destroyed,Stinson,108-3,No,1.0,Reciprocating,Personal,2.0,0.0,0.0,0.0,UNK,NaN
1,1962-07-19,"BRIDGEPORT, CA",United States,Destroyed,Piper,PA24-180,No,1.0,Reciprocating,Personal,4.0,0.0,0.0,0.0,UNK,NaN
2,1974-08-30,"Saltville, VA",United States,Destroyed,Cessna,172M,No,1.0,Reciprocating,Personal,3.0,NaN,NaN,NaN,IMC,NaN
3,1977-06-19,"EUREKA, CA",United States,Destroyed,Rockwell,112,No,1.0,Reciprocating,Personal,2.0,0.0,0.0,0.0,IMC,NaN
4,1979-08-02,"Canton, OH",United States,Destroyed,Cessna,501,No,NaN,NaN,Personal,1.0,2.0,NaN,0.0,VMC,NaN


## Remove all aircrafts that are not airplanes or helicopters

In [5]:
# Within the Aircraft Category variable, replace nan with "Unknown" and change "UNK" to "Unknown" 
df['Aircraft.Category'].fillna('Unknown', inplace = True)
df['Aircraft.Category'].replace({'UNK':'Unknown'}, inplace = True)

# Filter the df for airplanes, helicopters, and aircrafts we are unsure of 
df = df[(df['Aircraft.Category'] == 'Airplane') | (df['Aircraft.Category'] == 'Helicopter') | (df['Aircraft.Category'] == 'Unknown')]

# Display value counts of remaining aircrafts
df['Aircraft.Category'].value_counts(dropna=False)

Aircraft.Category
Unknown       56687
Airplane      27617
Helicopter     3440
Name: count, dtype: int64

Since there are many aircrafts that have an unknown aircraft category, we will use engine number and type to further filter for airplanes and helicopters.

In [6]:
# Check if Number.of.Engines = 0 corresponds to no Engine.Type
df[(df['Number.of.Engines']==0) & ((df['Engine.Type']!='NONE') & (df['Engine.Type'].notna())\
                                   & (df['Engine.Type'] != 'Unknown'))]

,Event.Date,Location,Country,Aircraft.damage,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Aircraft.Category
22648,1989-03-31,"GREAT HARBOR, Bahamas",Bahamas,Destroyed,Cessna,421B,No,0.0,Reciprocating,Unknown,1.0,0.0,0.0,1.0,VMC,Unknown
77176,2015-06-14,"Pell City, AL",United States,Substantial,WEATHERLY,201B,No,0.0,Reciprocating,Aerial Application,0.0,0.0,0.0,1.0,VMC,Airplane


In [7]:
# Replace Number.of.Engines = 0 with Number.of.Engines = Unknown if there is a recorded Engine.Type
replace_indx = list(df[(df['Number.of.Engines']==0) 
                    & ((df['Engine.Type']!='NONE') 
                       & (df['Engine.Type'].notna()) 
                       & (df['Engine.Type'] != 'Unknown'))].index)

df.loc[replace_indx, 'Number.of.Engines'] = 'Unknown'

In [8]:
# Drop all aircrafts with no engines 
drop_rows = df.loc[(df['Number.of.Engines']==0)].index
df = df.drop(drop_rows).reset_index(drop=True)

print(f"Dropped {len(drop_rows)} rows.")

Dropped 726 rows.


In [9]:
df['Engine.Type'].fillna('UNK', inplace = True)
df['Engine.Type'].replace({'UNK':'Unknown'}, inplace = True)

# Fill nan with Unknown 
df['Number.of.Engines'].fillna('Unknown', inplace = True)

In [10]:
# Drop aircrafts with engines uncommon for airplanes and helicopters 
drop_engines = ['NONE', 'LR', 'Hybrid Rocket']
df = df[df['Engine.Type'].apply(lambda x: x not in drop_engines)]

In [11]:
# Amongst remaining rows, drop cells with an unknown number of engines AND unknown engine type.
# We cannot identify whether these are airplanes, helicopters, or other.

drop_rows = df.loc[(df['Number.of.Engines']=='Unknown') & (df['Engine.Type']=='Unknown')].index
df = df.drop(drop_rows).reset_index(drop=True)

print(f"Dropped {len(drop_rows)} rows.")

Dropped 5242 rows.


In [12]:
df['Number.of.Engines'].value_counts(dropna=False)

Number.of.Engines
1.0        69043
2.0        11070
Unknown      750
3.0          482
4.0          429
8.0            1
6.0            1
Name: count, dtype: int64

In [13]:
df['Engine.Type'].value_counts(dropna=False)

Engine.Type
Reciprocating      69042
Turbo Shaft         3609
Turbo Prop          3391
Unknown             2534
Turbo Fan           2481
Turbo Jet            700
Geared Turbofan       12
Electric               7
Name: count, dtype: int64

In [14]:
print(f"Shape after dropping all but airplanes and helicopters: {df.shape}")

Shape after dropping all but airplanes and helicopters: (81776, 16)


## Data Cleaning

In [15]:
# Check if any of the injury columns have true values. If any do, fill nans with 0s. 
# If all columns are nans, assume the data was not logged and keep them as nans. 
injury_cols = ['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']
injury_data_exists = df[injury_cols].apply(lambda col: any(col), axis = 1)

for col in injury_cols:
    df.loc[injury_data_exists & df[col].isna(), col] = 0

We needed to have information about the amount of passengers that were on each incident. We used the information in the four columns related to injuries to extract it.

Aditionally we created percentages for the passengers that: Got injured, ended uninjured, and died.

In [16]:
#Obtaining the total of passengers
#Make note of dividing by zero situation in pecert.Injured and Percent.Died / Number of Passengers

df['Total.Passengers'] = df['Total.Fatal.Injuries'] + df['Total.Serious.Injuries'] + df['Total.Minor.Injuries']+ df['Total.Uninjured']
df['Total.Injured'] = df['Total.Fatal.Injuries'] + df['Total.Serious.Injuries'] + df['Total.Minor.Injuries']

#Creating Percentages for every kind of situation.
# Note that we had to account for division by 0 where the plane had no passengers.

df['Percent.Injured'] = (df['Total.Injured'] / df['Total.Passengers']) * 100
df['Percent.Uninjured'] = (df['Total.Uninjured'] / df['Total.Passengers']) * 100
df['Percent.Died'] = (df['Total.Fatal.Injuries'] / df['Total.Passengers']) * 100
df.loc[df['Total.Passengers'] == 0, ['Percent.Injured','Percent.Died','Percent.Uninjured']] = 0

In [18]:
# We decided to remove everything that is not Amateur Built.
# Remove rows where Amateur.Built is Yes or NaN, then remove Amateur.Built column

df = df.drop(df.loc[(df['Amateur.Built']=='Yes') |( df['Amateur.Built'].isna())].index)
df.reset_index(drop = True, inplace = True)
df.drop(columns = 'Amateur.Built', inplace = True)
df.head()

,Event.Date,Location,Country,Aircraft.damage,Make,Model,Number.of.Engines,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Aircraft.Category,Total.Passengers,Total.Injured,Percent.Injured,Percent.Uninjured,Percent.Died
0,1948-10-24,"MOOSE CREEK, ID",United States,Destroyed,Stinson,108-3,1.0,Reciprocating,Personal,2.0,0.0,0.0,0.0,UNK,Unknown,2.0,2.0,100.000000,0.000000,100.0
1,1962-07-19,"BRIDGEPORT, CA",United States,Destroyed,Piper,PA24-180,1.0,Reciprocating,Personal,4.0,0.0,0.0,0.0,UNK,Unknown,4.0,4.0,100.000000,0.000000,100.0
2,1974-08-30,"Saltville, VA",United States,Destroyed,Cessna,172M,1.0,Reciprocating,Personal,3.0,0.0,0.0,0.0,IMC,Unknown,3.0,3.0,100.000000,0.000000,100.0
3,1977-06-19,"EUREKA, CA",United States,Destroyed,Rockwell,112,1.0,Reciprocating,Personal,2.0,0.0,0.0,0.0,IMC,Unknown,2.0,2.0,100.000000,0.000000,100.0
4,1979-09-17,"BOSTON, MA",United States,Substantial,Mcdonnell Douglas,DC9,2.0,Turbo Fan,NaN,0.0,0.0,1.0,44.0,VMC,Airplane,45.0,1.0,2.222222,97.777778,0.0


In [19]:
# Clean Weather Condition
#Change the notation "IMC" and "VMC" to have a clear understanding of each condition.
df['Weather.Condition'].replace({'Unk':'Unknown','UNK':'Unknown','VMC':'Visual Meteorological Conditions',\
                                 'IMC':'Instrumental Meteorological Conditions'}, inplace = True)
df['Weather.Condition'].fillna('Unknown', inplace = True)
df['Weather.Condition'].value_counts(dropna=False)

Weather.Condition
Visual Meteorological Conditions          65672
Instrumental Meteorological Conditions     5596
Unknown                                    2446
Name: count, dtype: int64

In [20]:
df['Aircraft.Category'].value_counts(dropna=False)

Aircraft.Category
Unknown       48550
Airplane      22301
Helicopter     2863
Name: count, dtype: int64

In [21]:
# Replacing 'NaN' values in Aircraft Damage
df['Aircraft.damage'].fillna('Unknown', inplace = True)
df['Aircraft.damage'].value_counts(dropna = False)

Aircraft.damage
Substantial    54757
Destroyed      14862
Minor           2131
Unknown         1964
Name: count, dtype: int64

In [22]:
#Transform the Event Date into three separate Date columns that specify day, month and year of the incident.
#Dropping 'Event.Date' once the columns are created:
df['Event.Date'] = pd.to_datetime(df['Event.Date'])
df['Event.Day'] = df['Event.Date'].map(lambda x: x.day)
df['Event.Month'] = df['Event.Date'].map(lambda x: x.month)
df['Event.Month.Name'] = df['Event.Date'].map(lambda x: x.month_name())
df['Event.Year'] = df['Event.Date'].map(lambda x: x.year)
df.drop(['Event.Date'], axis=1, inplace=True)

### Working with the location

We decided to keep data from every country, so we could have more data to analyze the performance of the airplanes.

But, since we're not interested in the locations outside of the US, we rename all of the other countries "Foreign Country" to make the classification process easier.

For the Location Column, we just decided to kept the code for the corresponding State. All the ocurrences that were either in a foreign country or Unknown will be categorized as "Unknown/Foreign Location"

In [23]:
#Grouping all the Countries outside of the US as "Foreign Country"
df.loc[df['Country'] != 'United States', 'Country'] = 'Foreign Country'

In [24]:
#Filling the Null Values with "Unknown" the the Location Column
df['Location'] = df['Location'].fillna('Unknown')

# Adjusting the Column so it shows code of the corresponding State, if it's Unknown or outside of the US, label accordingly
df['Location'] = df['Location'].apply(lambda location: location.split(', ')[-1] if \
                                      len(location.split(', ')) > 1 and len(location.split(', ')[-1]) == 2 else \
                                      'Unknown/Foreign Location')

In [25]:
#Cleaning the 'Make' Column
#Create the list 'characters_to_remove' so we can drop any puctuation sign that can make our name differ.
characters_to_remove = ['(', ')', ',', '.', '%', '?','-']
# Fill 'NaN' values with Unknown.
df['Make'] = df['Make'].str.title().fillna('Unknown')
df['Make'] = df['Make'].map(lambda x: ''.join(char for char in x if char not in characters_to_remove))

#Based on further analysis, we found different names which certain variations and added words.
# 'names_var' dictionary is a way to unify names that have many variatios

names_var = {'Boeing': 'Boeing', 'Cirrus':'Cirrus','Airbus':'Airbus','Douglas':'Boeing', \
             'Air Tractor':'Air Tractor','Embraer':'Embraer','Bombardier':'Bombardier'}
for key, value in names_var.items():
    df.loc[df['Make'].str.contains(key), 'Make'] = value

df['Make'].loc[df['Make'].str.contains('Robinson Helicopter Company|Robinson Helicopter', case=False)] = 'Robinson'
    
#Eliminating all the rows which 'Make' involves anything helicopter related, since we're analyzing just airplanes.
#df = df[~df['Make'].str.contains('helicopter|copter|robinson', case=False)]

#Since there are many 'Makes' with a really low count of incidences, we keep just the top 50.
top_50_makes = df['Make'].value_counts().index[:50]
df = df[df['Make'].isin(top_50_makes)]

In [26]:
#Cleaning the 'Model' Column
df['Model'] = df['Model'].str.title().fillna('Unknown')
characters_to_remove = ['(', ')', '/', ' ', '-']
df['Model'] = df['Model'].map(lambda x: ''.join(char for char in x if char not in characters_to_remove))
df['Model'] = df['Model'].map(lambda x: x.lower())

In [27]:
#Replacing NaN values for 'Unknown in 'Purpose of flight':
df['Purpose.of.flight'].fillna('Unknown', inplace=True)

In [28]:
if generate_csvs == 1:   
    df.to_csv('data/data_clean.csv', index_label = 'index')